# 使用接口。并传入API_KEY

In [2]:
from mp_api.client import MPRester
API_KEY = "igRHy7zYOKzWD18jY76XtjbwEpRl6SoH"
mpr = MPRester(API_KEY)#创建MPRester对象进行认证

D:\anaconda3\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


查看一下，哪些是我们可以获取的fields

我们这里需要
```
material_id, structure,
band_gap,
bulk_modulus,
efermi,
formation_energy_per_atom,
```

In [3]:
list_of_available_fields = mpr.materials.summary.available_fields
print(list_of_available_fields)

['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites', 'types_of_magnetic_species', 'bulk_modulus', 'shear_modulus', 'universal_anisotropy', 'homogeneous_poisson', 'e_total', 'e_ionic', 'e_electronic',

# 任务1：查找并验证预测

查找mp-1000041的band_gap

在使用Xie的pre-trained模型中，预测的数值为：-2.1132540702819824

In [5]:
docs = mpr.materials.summary.search(material_ids=["mp-1000041"], fields=["band_gap"])
print(docs)

Retrieving SummaryDoc documents: 0it [00:00, ?it/s]

[]


查询不到任何数据，说明1000041的材料不在Material Project数据库中。

# 任务2：下载一个Si的示例材料cif文件，构建一个小数据集，预测+验证
这里选取的是mp-149编号,formation_energy_per_atom=0.0.

In [8]:
# 先查询他的结构数据
docs = mpr.materials.summary.search(material_ids=["mp-149"],
                                    fields= ["structure","formation_energy_per_atom"])
structure = docs[0].structure
print(docs)

Retrieving SummaryDoc documents: 100%|██████████| 1/1 [00:00<?, ?it/s]

[MPDataDoc<SummaryDoc>(
structure=Structure Summary
Lattice
    abc : 3.8492784033699095 3.8492794116013456 3.849278
 angles : 60.00001213094421 60.00000346645984 60.0000109754579
 volume : 40.32952684741405
      A : 3.333573 0.0 1.924639
      B : 1.111191 3.142924 1.924639
      C : 0.0 0.0 3.849278
    pbc : True True True
PeriodicSite: Si (3.889, 2.75, 6.736) [0.875, 0.875, 0.875]
PeriodicSite: Si (0.5556, 0.3929, 0.9623) [0.125, 0.125, 0.125],
formation_energy_per_atom=0.0,
fields_not_requested=['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cb

In [12]:
print(docs[0].formation_energy_per_atom)

0.0


保存结构的cif文件

In [10]:
structure.to("mp-149"+".cif")

D:\anaconda3\envs\pytorch\Lib\site-packages\pymatgen\io\cif.py:1764: FutureWarning: We strongly discourage using implicit binary/text `mode`, and this would not be allowed after 2025-06-01. I.e. you should pass t/b in `mode`.
  with zopen(filename, mode=mode) as file:


"# generated using pymatgen\ndata_Si\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   3.84927840\n_cell_length_b   3.84927941\n_cell_length_c   3.84927800\n_cell_angle_alpha   60.00001213\n_cell_angle_beta   60.00000347\n_cell_angle_gamma   60.00001098\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   Si\n_chemical_formula_sum   Si2\n_cell_volume   40.32952685\n_cell_formula_units_Z   2\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  Si  Si0  1  0.87500000  0.87500000  0.87500000  1\n  Si  Si1  1  0.12500000  0.12500000  0.12500000  1\n"

使用

```
python predict.py pre-trained/formation-energy-per-atom.pth.tar demo/mp-149
```
预测

结果如下：

```
Test: [0/1]     Time 7.108 (7.108)      Loss 0.0071 (0.0071)    MAE 0.089 (0.089)
 ** MAE 0.089

mp-149,0.0,0.08909666538238525
```

任务3：下载1000个数据文件，并将其整合成数据集，进行模型验证和训练。

分为以下几步：
1. 下载关于Si的1200个晶体structure cif文件；
2. field 所有晶体的formation_energy_per_atom和band_gap；
3. 用pre-trained模型对formation_energy_per_atom进行预测+search检验；
4. 在id_prop.csv中prop一项输入相应晶体的band_gap；
5. 800个用于train，200个用于val，200个用于test；
6. 用训练好的模型，对mp-419进行预测；
7. 用pre-trained模型对mp-419进行预测；
8. 查询mp-149的band_gap进行验证对比。

## step1：下载1200个晶体结构文件

In [14]:
docs = mpr.summary.search(elements=['Si'],fields=['material_id','structure','formation_energy_per_atom','band_gap'])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_5228\2986088491.py:1: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  docs = mpr.summary.search(elements=['Si'],fields=['material_id','structure','formation_energy_per_atom','band_gap'])
Retrieving SummaryDoc documents: 100%|██████████| 12673/12673 [01:03<00:00, 200.95it/s]


In [17]:
print(docs[:2])

[MPDataDoc<SummaryDoc>(
material_id=MPID(mp-1120447),
structure=Structure Summary
Lattice
    abc : 5.3863236377011114 4.99306507357568 6.8893479928519055
 angles : 69.43330295350191 109.94676695388326 96.5935365078348
 volume : 163.06526777122875
      A : 3.92838855 3.62881553 0.6418283100000001
      B : -3.19769998 2.03590595 3.24969239
      C : -0.5316016100000001 -3.91171428 5.64614974
    pbc : True True True
PeriodicSite: Si (3.022, 3.568, 2.057) [0.9773, 0.236, 0.1173]
PeriodicSite: Si (0.9641, 1.138, 6.496) [0.7985, 0.5565, 0.7395]
PeriodicSite: Si (0.7086, 0.8178, 2.815) [0.4217, 0.245, 0.3096]
PeriodicSite: Si (-0.3355, 1.244, 8.58) [0.7899, 0.9262, 0.8968]
PeriodicSite: Si (1.134, 2.967, 3.644) [0.7692, 0.5503, 0.2413]
PeriodicSite: Si (0.6029, -0.4367, 4.786) [0.4447, 0.2491, 0.6538]
PeriodicSite: Si (-0.1841, 3.218, 5.816) [0.7796, 0.9496, 0.3948]
PeriodicSite: Si (-1.131, -1.91, 4.357) [0.004084, 0.2548, 0.6246],
formation_energy_per_atom=0.37250966749999903,
band_gap=

In [20]:
for idoc in docs[:1200]:  # 保存前10个结构到cif文件
    print(idoc.material_id)
    idoc.structure.to(str(idoc.material_id)+".cif")

mp-1120447
mp-1244933
mp-1244971
mp-1244990
mp-1245041
mp-1245242
mp-1403870
mp-644693
mp-676011
mp-988210
mp-1079297
mp-1204046
mp-1056579
mp-999200
mp-92
mp-571520
mp-1199894
mp-1202745
mp-1094057
mp-1268191
mp-1196961
mp-34
mp-10649
mp-1120747
mp-1203790
mp-165
mp-1201492
mp-168
mp-1200830
mp-971661
mp-971662
mp-27
mp-149
mp-16220
mp-1014212
mp-1072544
mp-1001113
mp-1095269
mp-1204627
mp-2351841
mp-1079649
mp-109
mp-1524805
mp-1073686
mp-1073695
mp-1073702
mp-1073720
mp-1073721
mp-1073730
mp-1073733
mp-1073734
mp-1073750
mp-1073785
mp-1073809
mp-1073816
mp-1073819
mp-1073822
mp-1073857
mp-1073859
mp-1073861
mp-1074613
mp-1074619
mp-1074636
mp-1074644
mp-1074665
mp-1074668
mp-1074676
mp-1074681
mp-1074685
mp-1074700
mp-1074703
mp-1074733
mp-1074770
mp-1074772
mp-1244945
mp-1244965
mp-1244968
mp-1245026
mp-1391176
mp-1398433
mp-556319
mp-556588
mp-556788
mp-556880
mp-556985
mp-556994
mp-557004
mp-557873
mp-663422
mp-685105
mp-685184
mp-733790
mp-936187
mp-560708
mp-1073728
mp-1073745


编写id_prop.csv文件

In [22]:
import csv
from tqdm import tqdm
# 打开一个新的文件用于写入，如果文件不存在则创建
with open('Si_data_1200/id_prop.csv', 'w', newline='') as csvfile:
    # 创建一个csv写入器
    writer = csv.writer(csvfile)
    # 写入标题行（可选）
    # writer.writerow(['ID', 'Property'])
    # 遍历列表，写入每一行数据
    for idoc in tqdm(docs[:1200]):  # 保存前10个结构到cif文件
        # 写入ID和band_gap数据方便后续训练
        writer.writerow([idoc.material_id, idoc.band_gap])

100%|██████████| 1200/1200 [00:00<00:00, 57074.42it/s]


## step2：预测并查询Si1200数据集中的每一个formation...

In [ ]:
'''
python predict.py pre-trained/formation-energy-per-atom.pth.tar demo/Si_data_1200
'''

其结果如下：
```
 ** MAE 3.547

mp-1207203,0.0,-0.22629082202911377

mp-1077588,0.0,-0.17162573337554932

mp-644693,0.0,0.11410629749298096


```
查询这三个的对应数值

In [26]:
docs_3 = mpr.materials.summary.search(material_ids=["mp-1207203", "mp-1077588", "mp-644693"], fields=["material_id","formation_energy_per_atom"])
for idoc in docs_3:
    print(idoc.material_id,idoc.formation_energy_per_atom)

Retrieving SummaryDoc documents: 100%|██████████| 3/3 [00:00<?, ?it/s]

mp-1077588 0.022675953055556003
mp-1207203 -0.101274260554187
mp-644693 0.41511927999999904


| ID | Prediction | Search|
| --|----------------|------|
| mp-1207203 | -0.22629082202911377 | -0.101274260554187 |
| mp-1077588 | -0.17162573337554932 | -0.022675953055556003 |
| mp-644693 | 0.11410629749298096 | 0.41511927999999904 |